# Metadata CSV Preparation

This notebook adds HGNC symbols to BioMart CSV outputs and removes fully empty rows per Ensembl gene.

**Inputs**
- `data/raw/Human_TFs.csv`
- `data/intermediate/Metadata_CSVs/InterPro_Domains.csv`
- `data/intermediate/Metadata_CSVs/Pfam_Domains.csv`
- `data/intermediate/Metadata_CSVs/Transcript_Attributes.csv`
- `data/intermediate/Metadata_CSVs/External_References.csv`

**Outputs**
- `data/intermediate/Metadata_CSVs/*_cleaned.csv`

**Note**: Set `BHLH_PROJECT_ROOT` if running from a different working directory.


In [ ]:
import pandas as pd
from pathlib import Path

project_root = Path(__import__("os").getenv("BHLH_PROJECT_ROOT", ".")).resolve()

def p(*parts):
    return str(project_root.joinpath(*parts))


## 1) Add HGNC symbols to BioMart CSVs

In [ ]:
human_tfs = pd.read_csv(p("data", "raw", "Human_TFs.csv"))

csv_files = {
    "InterPro_Domains": p("data", "intermediate", "Metadata_CSVs", "InterPro_Domains.csv"),
    "Pfam_Domains": p("data", "intermediate", "Metadata_CSVs", "Pfam_Domains.csv"),
    "Transcript_Attributes": p("data", "intermediate", "Metadata_CSVs", "Transcript_Attributes.csv"),
}


In [ ]:
def add_hgnc_symbol(df: pd.DataFrame, hgnc_map: pd.DataFrame) -> pd.DataFrame:
    merged = df.merge(
        hgnc_map[["Ensembl ID", "HGNC symbol"]],
        left_on="ensembl_gene_id",
        right_on="Ensembl ID",
        how="left",
    ).drop(columns=["Ensembl ID"])
    cols = ["HGNC symbol"] + [c for c in merged.columns if c != "HGNC symbol"]
    return merged[cols]

for name, path in csv_files.items():
    df = pd.read_csv(path)
    df = add_hgnc_symbol(df, human_tfs)
    df.to_csv(path, index=False)
    print(f"Updated: {name}")


## 2) Remove fully empty rows per gene

In [ ]:
df_ext = pd.read_csv(p("data", "intermediate", "Metadata_CSVs", "External_References.csv"))
df_interpro = pd.read_csv(p("data", "intermediate", "Metadata_CSVs", "InterPro_Domains.csv"))
df_pfam = pd.read_csv(p("data", "intermediate", "Metadata_CSVs", "Pfam_Domains.csv"))
df_transcript = pd.read_csv(p("data", "intermediate", "Metadata_CSVs", "Transcript_Attributes.csv"))

frames = {
    "External_References": df_ext,
    "InterPro_Domains": df_interpro,
    "Pfam_Domains": df_pfam,
    "Transcript_Attributes": df_transcript,
}

key_columns = ["HGNC symbol", "ensembl_gene_id", "ensembl_transcript_id"]

def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    unique_ids = df["ensembl_gene_id"].unique()
    for ens_id in unique_ids:
        subset_df = df[df["ensembl_gene_id"] == ens_id]
        if len(subset_df) > 1:
            non_key_columns = [c for c in df.columns if c not in key_columns]
            empty_rows = subset_df[non_key_columns].isna().all(axis=1)
            if empty_rows.any():
                df = df.drop(subset_df[empty_rows].index)
    return df

for name, df in frames.items():
    cleaned = clean_dataframe(df)
    cleaned.to_csv(p("data", "intermediate", "Metadata_CSVs", f"{name}_cleaned.csv"), index=False)
    print(f"Saved cleaned {name}: {cleaned['ensembl_gene_id'].nunique()} unique Ensembl IDs")


## Exploratory analysis (optional)
These checks are optional and can be skipped in a production run.


In [ ]:
# Load cleaned files for exploratory analysis
cleaned_ext = pd.read_csv(p("data", "intermediate", "Metadata_CSVs", "External_References_cleaned.csv"))
cleaned_interpro = pd.read_csv(p("data", "intermediate", "Metadata_CSVs", "InterPro_Domains_cleaned.csv"))
cleaned_pfam = pd.read_csv(p("data", "intermediate", "Metadata_CSVs", "Pfam_Domains_cleaned.csv"))
cleaned_transcript = pd.read_csv(p("data", "intermediate", "Metadata_CSVs", "Transcript_Attributes_cleaned.csv"))

print("External_References:", cleaned_ext.shape)
print("InterPro_Domains:", cleaned_interpro.shape)
print("Pfam_Domains:", cleaned_pfam.shape)
print("Transcript_Attributes:", cleaned_transcript.shape)


In [ ]:
# Preview
cleaned_interpro.head(20)


In [ ]:
print(cleaned_interpro["interpro_start"].unique())
print(cleaned_interpro["interpro_start"].nunique())


### Most frequent domain IDs

In [ ]:
from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sns

# Optional SMART/CDD if available
smart_path = Path(p("data", "intermediate", "Metadata_CSVs", "SMART_Domains_cleaned.csv"))
cdd_path = Path(p("data", "intermediate", "Metadata_CSVs", "CDD_Domains_cleaned.csv"))

cleaned_smart = pd.read_csv(smart_path) if smart_path.exists() else None
cleaned_cdd = pd.read_csv(cdd_path) if cdd_path.exists() else None

interpro_counts = cleaned_interpro["interpro"].value_counts().head(10)
pfam_counts = cleaned_pfam["pfam"].value_counts().head(10)
smart_counts = cleaned_smart["smart"].value_counts().head(10) if cleaned_smart is not None else None
cdd_counts = cleaned_cdd["cdd"].value_counts().head(10) if cleaned_cdd is not None else None

fig, axes = plt.subplots(2, 2, figsize=(18, 12))

sns.barplot(x=interpro_counts.values, y=interpro_counts.index, ax=axes[0,0], palette="Blues_r")
axes[0,0].set_xlabel("Frequency")
axes[0,0].set_ylabel("InterPro ID")
axes[0,0].set_title("Top 10 InterPro Domains")

sns.barplot(x=pfam_counts.values, y=pfam_counts.index, ax=axes[0,1], palette="Oranges_r")
axes[0,1].set_xlabel("Frequency")
axes[0,1].set_ylabel("Pfam ID")
axes[0,1].set_title("Top 10 Pfam Domains")

if smart_counts is not None:
    sns.barplot(x=smart_counts.values, y=smart_counts.index, ax=axes[1,0], palette="Greens_r")
    axes[1,0].set_xlabel("Frequency")
    axes[1,0].set_ylabel("SMART ID")
    axes[1,0].set_title("Top 10 SMART Domains")
else:
    axes[1,0].set_title("SMART Domains (not available)")
    axes[1,0].axis("off")

if cdd_counts is not None:
    sns.barplot(x=cdd_counts.values, y=cdd_counts.index, ax=axes[1,1], palette="Purples")
    axes[1,1].set_xlabel("Frequency")
    axes[1,1].set_ylabel("CDD ID")
    axes[1,1].set_title("Top 10 CDD Domains")
else:
    axes[1,1].set_title("CDD Domains (not available)")
    axes[1,1].axis("off")

plt.tight_layout()
plt.show()


### Domain length distributions

In [ ]:
cleaned_interpro["domain_length"] = cleaned_interpro["interpro_end"] - cleaned_interpro["interpro_start"]
cleaned_pfam["domain_length"] = cleaned_pfam["pfam_end"] - cleaned_pfam["pfam_start"]

if cleaned_smart is not None:
    cleaned_smart["domain_length"] = cleaned_smart["smart_end"] - cleaned_smart["smart_start"]

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

sns.histplot(cleaned_interpro["domain_length"].dropna(), bins=60, kde=True, ax=axes[0])
axes[0].set_xlabel("Domain length")
axes[0].set_ylabel("Frequency")
axes[0].set_xlim(0, 500)
axes[0].set_title("InterPro domain lengths")

sns.histplot(cleaned_pfam["domain_length"].dropna(), bins=60, kde=True, ax=axes[1], color="orange")
axes[1].set_xlabel("Domain length")
axes[1].set_xlim(0, 400)
axes[1].set_ylim(0, 250)
axes[1].set_title("Pfam domain lengths")

if cleaned_smart is not None:
    sns.histplot(cleaned_smart["domain_length"].dropna(), bins=60, kde=True, ax=axes[2], color="green")
    axes[2].set_xlabel("Domain length")
    axes[2].set_xlim(0, 200)
    axes[2].set_ylim(0, 150)
    axes[2].set_title("SMART domain lengths")
else:
    axes[2].set_title("SMART domains (not available)")
    axes[2].axis("off")

plt.tight_layout()
plt.show()


### Domains per gene

In [ ]:
pfam_counts = cleaned_pfam.groupby("HGNC symbol")["pfam"].nunique()
interpro_counts = cleaned_interpro.groupby("HGNC symbol")["interpro"].nunique()
smart_counts = cleaned_smart.groupby("HGNC symbol")["smart"].nunique() if cleaned_smart is not None else None

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

sns.histplot(pfam_counts, bins=20, kde=True, ax=axes[0])
axes[0].set_title("Pfam domains per gene")
axes[0].set_xlabel("Number of Pfam domains")
axes[0].set_ylabel("Frequency")

sns.histplot(interpro_counts, bins=20, kde=True, ax=axes[1], color="orange")
axes[1].set_title("InterPro domains per gene")
axes[1].set_xlabel("Number of InterPro domains")
axes[1].set_ylabel("Frequency")

if smart_counts is not None:
    sns.histplot(smart_counts, bins=20, kde=True, ax=axes[2], color="green")
    axes[2].set_title("SMART domains per gene")
    axes[2].set_xlabel("Number of SMART domains")
    axes[2].set_ylabel("Frequency")
else:
    axes[2].set_title("SMART domains (not available)")
    axes[2].axis("off")

plt.tight_layout()
plt.show()


### bHLH domain length (InterPro IPR011598)

In [ ]:
cleaned_interpro["interpro_start"] = pd.to_numeric(cleaned_interpro["interpro_start"], errors="coerce")
cleaned_interpro["interpro_end"] = pd.to_numeric(cleaned_interpro["interpro_end"], errors="coerce")

ipr011598_df = cleaned_interpro[cleaned_interpro["interpro"] == "IPR011598"].copy()
ipr011598_df["domain_length"] = ipr011598_df["interpro_end"] - ipr011598_df["interpro_start"] + 1

plt.figure(figsize=(8, 5))
sns.histplot(ipr011598_df["domain_length"], bins=30, kde=True)
plt.xlabel("Domain length (aa)")
plt.ylabel("Frequency")
plt.title("Length distribution of IPR011598 domains")
plt.show()


### Genes with the most domains

In [ ]:
top_pfam = pfam_counts.nlargest(15)
top_interpro = interpro_counts.nlargest(15)
top_smart = smart_counts.nlargest(15) if smart_counts is not None else None

fig, axes = plt.subplots(1, 3, figsize=(20, 7))

sns.barplot(x=top_pfam.values, y=top_pfam.index, ax=axes[0], palette="Blues_r")
axes[0].set_title("Top 10 genes with most Pfam domains")
axes[0].set_xlabel("Number of Pfam domains")
axes[0].set_ylabel("Gene ID")

sns.barplot(x=top_interpro.values, y=top_interpro.index, ax=axes[1], palette="Oranges_r")
axes[1].set_title("Top 10 genes with most InterPro domains")
axes[1].set_xlabel("Number of InterPro domains")
axes[1].set_ylabel("Gene ID")

if top_smart is not None:
    sns.barplot(x=top_smart.values, y=top_smart.index, ax=axes[2], palette="Greens_r")
    axes[2].set_title("Top 10 genes with most SMART domains")
    axes[2].set_xlabel("Number of SMART domains")
    axes[2].set_ylabel("Gene ID")
else:
    axes[2].set_title("SMART domains (not available)")
    axes[2].axis("off")

plt.tight_layout()
plt.show()


### Isoform counts per gene

In [ ]:
isoform_counts = cleaned_transcript.groupby("HGNC symbol")["ensembl_transcript_id"].nunique()

plt.figure(figsize=(8, 5))
sns.histplot(isoform_counts, bins=30, kde=True)
plt.xlabel("Number of isoforms per gene")
plt.ylabel("Frequency")
plt.title("Distribution of isoforms per gene")
plt.show()


### Genes with the highest number of isoforms

In [ ]:
top_isoforms = isoform_counts.nlargest(10)

plt.figure(figsize=(8, 5))
sns.barplot(x=top_isoforms.values, y=top_isoforms.index, palette="viridis")
plt.xlabel("Number of isoforms")
plt.ylabel("Gene ID")
plt.title("Top 10 genes with the most isoforms")
plt.show()
